In [2]:
import pandas as pd
import numpy as np
import os
import io
import gzip


1. Lectura de los archivos en raw, y me creo los archivos parquet que voy a utilizar

In [3]:
df = pd.read_csv('../../data/raw/train.csv')
df_test_nolabel = pd.read_csv('../../data/raw/test_nolabel.csv')
df_sample_submission= pd.read_csv('../../data/raw/sample_submission.csv')

df_train = df_test_nolabel
#df_train = pd.read_parquet('../data/agregado_diario_2019-2021.parquet')

In [4]:
df_train.head()

,id,LoanNr_ChkDgt,Name,City,State,Bank,BankState,ApprovalDate,ApprovalFY,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementDate,DisbursementGross,BalanceGross
0,6b7ce9ec161,7996984008,SPRINGVALLEY ACCOUNTING SERVIC,HOLLAND,OH,CITIZENS BANK NATL ASSOC,RI,26-Oct-04,2005,9,1.0,0,9,1,2,Y,N,30-Nov-04,"$205,236.00",$0.00
1,96123015731,3675194008,FULL SCALE DEVELOPMENT CORPORA,WORTHINGTON,OH,KEYBANK NATIONAL ASSOCIATION,OH,4-Apr-00,2000,15,1.0,4,15,1,1,Y,N,31-May-00,"$51,000.00",$0.00
2,b2c5181ac5b,5995734002,KIRBY SAND & GRAVEL,UPPER SANDUSKY,OH,JPMORGAN CHASE BANK NATL ASSOC,IL,7-Jan-03,2003,13,1.0,0,13,1,2,Y,N,28-Feb-03,"$50,000.00",$0.00
3,e6cb54a9e6a,7630324004,BLUE ECLIPSE INC,DAYTON,OH,JPMORGAN CHASE BANK NATL ASSOC,IL,21-Jul-04,2004,28,2.0,19,28,1,1,0,N,30-Sep-04,"$131,600.00",$0.00
4,8eddf83466e,4304173001,"CAR-PEC GRAPHICS, INC.",AKRON,OH,CASCADE CAPITAL CORPORATION,OH,15-Nov-90,1991,20,1.0,17,3,1,0,N,N,13-Nov-91,"$139,000.00",$0.00


In [34]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3284 entries, 0 to 3283
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 3284 non-null   object 
 1   LoanNr_ChkDgt      3284 non-null   int64  
 2   Name               3284 non-null   object 
 3   City               3284 non-null   object 
 4   State              3284 non-null   object 
 5   Bank               3279 non-null   object 
 6   BankState          3279 non-null   object 
 7   ApprovalDate       3284 non-null   object 
 8   ApprovalFY         3284 non-null   object 
 9   NoEmp              3284 non-null   int64  
 10  NewExist           3284 non-null   float64
 11  CreateJob          3284 non-null   int64  
 12  RetainedJob        3284 non-null   int64  
 13  FranchiseCode      3284 non-null   int64  
 14  UrbanRural         3284 non-null   int64  
 15  RevLineCr          3280 non-null   object 
 16  LowDoc             3275 

In [35]:
df_train.isnull().sum()

id                   0
LoanNr_ChkDgt        0
Name                 0
City                 0
State                0
Bank                 5
BankState            5
ApprovalDate         0
ApprovalFY           0
NoEmp                0
NewExist             0
CreateJob            0
RetainedJob          0
FranchiseCode        0
UrbanRural           0
RevLineCr            4
LowDoc               9
DisbursementDate     7
DisbursementGross    0
BalanceGross         0
dtype: int64

2. Cambiar los valores correspondientes.

La gran mayoría de los valores son de tipo objet, int64 y float64, tienen que ser acorde a sus valores.

Primero cambio los valores de las fechas, que son Approval Date y DisbursementDate que están puesto con letra escrita en vez de en el formato de numero.
Después cambiamos los valores que están raros y no se adaptan

* *DisbursementDate* => transformar la fecha
* *Name* => String, Si no sabemos lo ponemos Unknown
* *City* => String, Si no sabemos lo ponemos Unknown
* *State* => Son todosde tipo OH
* *Bank* => Si quitamos los None es de tipo String, Si no sabemos lo ponemos Unknown
* *BankState* => Si quitamos los None es de tipo String, Si no sabemos lo ponemos Unknown
* *NewExist* => 2 / 1 / 0, int, y poner la media donde sea nulo; 

    1 = Existing business, 2 = New business; 

    Si no lo sabemos ponemos vieja empresa que es la mayoría, es decir, que ya existía la empresa, 0. 

    Por lo que corresponde con 0 = Existing y 1 = New
* *RevLineCr* => array(['N', 'Y', '0', 'T', None, '`'], dtype=object, 

    Hay que limpiarlo, sabiendo que tiene que ser Y = Yes, N = No, para el que no sepamos ponemos la mayoría; después ponemos True => 1 y False => 0
* *LowDoc* => array(['N', 'Y', None, 'A', '0', 'S', 'R', 'C'], dtype=object)

    Tiene que ser Y = Yes, N = No, para el que no sepamos ponemos la mayoría; después ponemos True => 1 y False => 0
* *DisbursementGross* => Quitar el dolar y hacerlo double
* *BalanceGross* => Quitar el dolar y hacerlo double y donde sea nulo

In [ ]:
df

<>:31: SyntaxWarning: invalid escape sequence '\$'
<>:32: SyntaxWarning: invalid escape sequence '\$'
<>:35: SyntaxWarning: invalid escape sequence '\d'
<>:31: SyntaxWarning: invalid escape sequence '\$'
<>:32: SyntaxWarning: invalid escape sequence '\$'
<>:35: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_171188/177652060.py:31: SyntaxWarning: invalid escape sequence '\$'
  df_train2['DisbursementGross'] = df_train2['DisbursementGross'].replace('[\$,]', '', regex=True).astype(float)
/tmp/ipykernel_171188/177652060.py:32: SyntaxWarning: invalid escape sequence '\$'
  df_train2['BalanceGross'] = df_train2['BalanceGross'].replace('[\$,]', '', regex=True).astype(float)
/tmp/ipykernel_171188/177652060.py:35: SyntaxWarning: invalid escape sequence '\d'
  df_train2['ApprovalFY'] = df_train2['ApprovalFY'].astype(str).str.extract('(\d+)', expand=False)
/tmp/ipykernel_171188/177652060.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling ba

2. Cambiar los valores correspondientes.

La gran mayoría de los valores son de tipo objet, int64 y float64, tienen que ser acorde a sus valores.

Primero cambio los valores de las fechas, que son Approval Date y DisbursementDate que están puesto con letra escrita en vez de en el formato de numero.
Después cambiamos los valores que están raros y no se adaptan

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22835 entries, 0 to 22834
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 22835 non-null  object 
 1   LoanNr_ChkDgt      22835 non-null  int64  
 2   Name               22834 non-null  object 
 3   City               22834 non-null  object 
 4   State              22835 non-null  object 
 5   Bank               22813 non-null  object 
 6   BankState          22813 non-null  object 
 7   ApprovalDate       22835 non-null  object 
 8   ApprovalFY         22835 non-null  int64  
 9   NoEmp              22835 non-null  int64  
 10  NewExist           22833 non-null  float64
 11  CreateJob          22835 non-null  int64  
 12  RetainedJob        22835 non-null  int64  
 13  FranchiseCode      22835 non-null  int64  
 14  UrbanRural         22835 non-null  int64  
 15  RevLineCr          22744 non-null  object 
 16  LowDoc             227

* *DisbursementDate* => transformar la fecha
* *Name* => String, Si no sabemos lo ponemos Unknown
* *City* => String, Si no sabemos lo ponemos Unknown
* *State* => Son todosde tipo OH
* *Bank* => Si quitamos los None es de tipo String, Si no sabemos lo ponemos Unknown
* *BankState* => Si quitamos los None es de tipo String, Si no sabemos lo ponemos Unknown
* *NewExist* => 2 / 1 / 0, int, y poner la media donde sea nulo; 

    1 = Existing business, 2 = New business; 

#Limpiamos los dólares y lo ponemos en tipo float
df_train2['DisbursementGross'] = df_train2['DisbursementGross'].replace('[\$,]', '', regex=True).astype(float)
df_train2['BalanceGross'] = df_train2['BalanceGross'].replace('[\$,]', '', regex=True).astype(float)

####################### Limpiar la columna 'ApprovalFY' para extraer solo los dígitos (los años)
df_train2['ApprovalFY'] = df_train2['ApprovalFY'].astype(str).str.extract('(\d+)', expand=False)

# Convertir a numérico (opcional, pero útil para operaciones posteriores)
df_train2['ApprovalFY'] = pd.to_numeric(df_train2['ApprovalFY'], errors='coerce')

df_train2.to_csv('../../data/processed/df_test_nolabel.csv', index=False)
# df_train2.to_csv('../../data/processed/df_train.csv', index=False)


In [37]:
# Para guardar el archivo como df_train.csv

# Conversion de datos
df_train = conversion(df)

# Formateo
df_train_fitted = formatear(df_train)

output_path = "../../data/processed/train_processed_v3.csv"
df_train_fitted.to_csv(output_path, index=False)
print("Archivo guardado en:", output_path)

id                    0
LoanNr_ChkDgt         0
Name                  0
City                  0
State                 0
Bank                  0
BankState             0
ApprovalDate          0
ApprovalFY            0
NoEmp                 0
NewExist              3
CreateJob             0
RetainedJob           0
FranchiseCode         0
UrbanRural            0
RevLineCr             4
LowDoc               18
DisbursementDate      7
DisbursementGross     0
BalanceGross          0
dtype: int64

In [38]:
df_train2.BalanceGross.value_counts(dropna=False)

BalanceGross
0.0    3284
Name: count, dtype: int64

In [7]:
df_train2.FranchiseCode.unique()

array([    1,     0, 57000, 34650, 39237, 52300, 24850, 61901, 26556,
       26650, 50564, 84535, 20487, 81612, 74789, 43579, 65160, 50965,
       17998, 88875, 21780, 68020, 91283, 68451, 78760, 79140, 55960,
       53651, 38605, 35711, 65100, 78950, 44081, 63345, 10489, 52875,
       58615, 73675, 10528, 53151, 45915, 70150, 63810, 35702, 73740,
       40665, 68060, 22470, 17205, 17590, 67715, 56575, 24902, 36260,
       79902, 66486, 72590, 35605, 77875, 10776, 23379, 68250, 51720,
       35307, 66910, 47751, 62360, 43552, 91170, 31415, 35706,   407,
       91350, 48550, 20100, 84060, 35705, 77174, 15713, 38610, 89020,
       34849, 24419,  7990, 81601, 53650, 36680, 65140, 30450, 14348,
       17256, 73525,  6305,  9235, 80979, 10618, 21420, 52000,  9450,
       10465, 29900, 36355,  1560, 75900, 20970, 49952, 10559, 44066,
       66320, 67750, 42600, 10481, 16920,  4516, 63010, 25360, 66345,
       26879, 60525, 61897, 34900, 70670, 25650, 60590,  8435, 80360,
       41427, 18925,